In [5]:
# Import the SPICE module
import spiceypy

# Import datetime and get the current time stamp
import datetime

# get today's date
date_today = datetime.datetime.today()

# convert the datetime to a string, replacing the time with midnight
date_today = date_today.strftime('%Y-%m-%dT00:00:00')

# Load the spice kernels for leapseconds and for the planets
spiceypy.furnsh('../kernels/lsk/naif0012.tls')
spiceypy.furnsh('../kernels/spk/de432s.bsp')

# Compute the Ephemeris Time
et_today_midnight = spiceypy.utc2et(date_today)

# Compute the state vector of the Earth w.r.t. the Sun
earth_state_wrt_sun, earth_sun_light_time = spiceypy.spkgeo(targ=399,
                                                            et=et_today_midnight,
                                                            ref='ECLIPJ2000',
                                                            obs=10)

In [6]:
# The state vector is 6 dimensional: x,y,z in km and the corresponding velocities in km/s
print(f'State vector of the Earth w.r.t. the Sun for {date_today} (midnight):\n'
      + f'{earth_state_wrt_sun}')

State vector of the Earth w.r.t. the Sun for 2021-10-06T00:00:00 (midnight):
[ 1.45937499e+08  3.27940122e+07 -2.45570856e+03 -7.01712100e+00
  2.89650470e+01 -6.82506555e-04]


In [7]:
import numpy as np

# Convert list to numpy array
earth_state_wrt_sun = np.array(earth_state_wrt_sun)

# Compute the distance
earth_sun_distance = np.linalg.norm(earth_state_wrt_sun[:3])

# First, we compute the actual orbital speed of the Earth around the Sun
earth_orb_speed_wrt_sun = np.linalg.norm(earth_state_wrt_sun[3:])

# It's around 30 km/s
print(f'Current orbital speed of the Earth around the Sun in km/s: {earth_orb_speed_wrt_sun}')

Current orbital speed of the Earth around the Sun in km/s: 29.80291819056653


In [8]:
# Now let's compute the theoretical expectation. First, we load a pck file that contains
# miscellanoeus information, like the G*M values for different objects

# First, load the kernel
spiceypy.furnsh('../kernels/pck/gm_de431.tpc')
_, GM_SUN = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)

# Now compute the orbital speed
v_orb_func = lambda gm, r: np.sqrt(gm/r)
earth_orb_speed_wrt_sun_theory = v_orb_func(GM_SUN[0], earth_sun_distance)

# Print the result
print(f'Theoretical orbital speed of the Earth around the Sun in km/s: '
      + f'{earth_orb_speed_wrt_sun_theory}')

Theoretical orbital speed of the Earth around the Sun in km/s: 29.786795676472426


In [9]:
# A second check:
# The angular difference between the autumn equinox and today's position vector of the Earth
# (in this tutorial October) should be in degrees the number of days passed the 22th September.
# Again please note: we use the "today" function to determine the Earth's state vector.
# Now the "autumn vector" is simpley (1, 0, 0) in ECLIPJ2000 and we use this as a quick and simple
# rough estimation / computation

# Position vector
earth_position_wrt_sun = earth_state_wrt_sun[:3]

# Normalize it
earth_position_wrt_sun_normed = earth_position_wrt_sun / earth_sun_distance

# Define the "autumn vector" of the Earth
earth_position_wrt_sun_normed_autumn = np.array([1.0, 0.0, 0.0])

In [10]:
ang_dist_deg = np.degrees(np.arccos(np.dot(earth_position_wrt_sun_normed,
                                           earth_position_wrt_sun_normed_autumn)))
print(f"Angular distance between autumn and today's position in degrees {date_today}: "
      + f"{ang_dist_deg}")

Angular distance between autumn and today's position in degrees 2021-10-06T00:00:00: 12.664715673560407
